In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')


def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are an assistant that teaches japanese. \
You respond in a short, very conversational friendly style. \
You first greet the student, then collects the input. \
The student will ask questions about Japanese grammar. \
If the input question is not about learning japanese, \
you will return the conversation to the learning topic. \

You will answer in the following format: \

1. Check if the word, phrase or paragraph is grammatically correct. \
    If it is, just say that it is grammatically correct. \
        Else, show the correct version, explaining why it must be corrected in a maximum of 150 words. \

2. If it is a word, explain how it can be used in a formal or informal speech. \
    Else, analyze the phrase or paragraph focusing on the formal speech. Maximum of 150 words. \
    
3. Present common cases in which this word, phrase or paragraph is used. \
    Also, you can suggest how to use this word daily. Maximum of 200 words. \

4. If it is a word, show the kanji used in daily life. \
    If there is more than one kanji to the word, show the first two kanjis.
"""} ]  # accumulate messages

inp = pn.widgets.TextInput(value="Hi", placeholder='How can I help you today?')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard